# CSC 522 Project 
_by Team 21: Raphael Phillips (rjphill4), Sogolsadat Mansouri (smansou2), Rithik Jain (rjain25), and Neeloy Gomes (ntgomes)_

This project will involve developing a machine learning model using supervised learning to predict future exchange rates for a specific currency pair, based on historical exchange rate data and other relevant features.


In [11]:
%run pre_processing.py

usdinr_df = pd.read_csv('../data/USDINRX.csv')
forex_df = pd.read_csv('../data/forex.csv')

cleaned_usdinr_df = pre_p_usdinr(usdinr_df)
cleaned_forex_df = pre_p_forex(forex_df)

print('Cleaned ' + str(len(forex_df.index) - len(cleaned_forex_df.index)) + ' rows from data/forex.csv')
print('Cleaned ' + str(len(usdinr_df.index) - len(cleaned_usdinr_df.index)) + ' rows from data/USDINRX.csv')


TypeError: Index.drop() got an unexpected keyword argument 'columns'

In [ ]:
%run exploratory_analysis.py

# do_analysis()